In [1]:
import json
import requests
import citipy
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import gmaps
from config import gkey
cities_sample = pd.read_csv("../WeatherPy/cities_sample.csv")
cities_sample = cities_sample.rename(columns = {'Unnamed: 0': 'Country ID'})
gmaps.configure(api_key = gkey)

In [2]:
cities_sample.head()

,Country ID,Country,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,28204,PH,Gibato,11.225100,122.814300,87.71,57.0,16.0,13.20
1,43856,US,Braintree,42.222222,-71.000000,34.43,86.0,1.0,5.95
2,29254,PH,New Visayas,7.523333,125.623333,84.20,74.0,75.0,10.29
3,33169,RO,Goicea,43.916667,23.616667,59.00,58.0,96.0,10.29
4,21904,JP,Ujiie,36.683333,139.966667,64.00,43.0,24.0,11.54


In [3]:
locations = cities_sample[['Latitude', 'Longitude']]
max_humid = cities_sample['Humidity (%)'].max()

In [4]:
fig = gmaps.figure()
#Create a heat map that displays the humidity for every city from the part I of the homework.

humidity_layer = gmaps.heatmap_layer(locations, weights = cities_sample['Humidity (%)'], max_intensity = max_humid, point_radius = 10)
fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
#Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

ideal_temp = cities_sample.loc[((cities_sample['Temperature (F)'] < 80) & (cities_sample['Temperature (F)'] > 70))]
ideal_wind = ideal_temp.loc[cities_sample['Wind Speed (mph)'] < 10]
ideal_locations = ideal_wind.loc[ideal_wind['Cloudiness (%)'] == 0]
ideal_locations

,Country ID,Country,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
8,22125,KY,Bodden Town,19.283333,-81.250000,79.66,76.0,0.0,3.00
274,10275,DO,Santo Domingo,18.466667,-69.900000,75.99,83.0,0.0,2.75
338,10608,EG,Hurghada,27.238889,33.836111,75.20,41.0,0.0,8.05


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?key=" + gkey
hotel_df = pd.DataFrame([{'Name': "",
                        'City': "",
                        'Country': "",
                        'Latitude': "",
                        'Longitude': ""}], index = ideal_locations.index.values)
for index, row in ideal_locations.iterrows():
    lat, lng = row['Latitude'], row['Longitude']
    hotel_results = requests.get(base_url + f"&location={lat},{lng}&rankby=distance&keyword=hotel").json()['results']
    try:
        hotel_df.loc[index, 'Name'] = hotel_results[0]['name']
        hotel_df.loc[index, 'City'] = hotel_results[0]['vicinity']
        hotel_df.loc[index, 'Country'] = hotel_results[0]['plus_code']['compound_code'].split(',')[-1]
        hotel_df.loc[index, 'Latitude'] = hotel_results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Longitude'] = hotel_results[0]['geometry']['location']['lng']
    except:
        print(f"No hotels near {lat}, {lng}.")
hotel_df

,Name,City,Country,Latitude,Longitude
8,Sea Palm Villas,258 Bodden Town Rd,Cayman Islands,19.2783,-81.2489
274,Hotel Hostal San Francisco de Asis,"Av. Pasteur, Santo Domingo",Dominican Republic,18.4665,-69.8988
338,Bel Air Azur Resort,First Hurghada,Egypt,27.2417,33.8434


In [7]:
hotel_locations = [(row['Latitude'], row['Longitude']) for index, row in hotel_df.iterrows()]
info_box_template = """<dt>Name:</dt> {Name} <dt>City:</dt> {City} <dt>Country:</dt> {Country}"""
hotel_info = [{'Name': row['Name'],
              'City': row['City'],
              'Country': row['Country']} for index, row in hotel_df.iterrows()]

In [8]:
hotel_layer = gmaps.marker_layer(hotel_locations, info_box_content = [info_box_template.format(**hotel) for hotel in hotel_info])
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))